In [12]:
# Load R language extension into Jupyter environment
%load_ext rpy2.ipython
# Load R project dependency management environment
%R renv::load()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
- Project '/content' loaded. [renv 1.0.11]


'/content'


In [21]:

from google.colab import drive
drive.mount('/content/drive')

import os
project_path = "/content/drive/MyDrive/PopBERT/"
os.chdir(project_path)

import sys
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from itertools import combinations # Import combination function tool from Python standard library

import numpy as np
import pandas as pd

# Import R language interaction related libraries
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

# Import machine learning evaluation metrics and statistical tools
from sklearn.metrics import f1_score
from statsmodels.stats import inter_rater

# Import local src module and its submodules
import src
import src.bert.dataset.strategies as strategies
import src.db.connect
from src.bert.dataset import PBertDataset

In [25]:
dataset = PBertDataset.from_disk(
    path=src.PATH / "data/labeled_data/full.csv.zip",
    exclude_coders=[],
    label_strategy=strategies.MLMin1PopIdeol(output_fmt="single_task"),
)

In [26]:
def pairwise_f1_agreement(df):
    scores = []
    for i, j in combinations(df.columns, r=2):
        coder1 = df[i].apply(pd.Series).to_numpy()
        coder2 = df[j].apply(pd.Series).to_numpy()
        score = f1_score(coder1, coder2, average="binary")
        scores.append(score)

    return np.mean(scores)

Evaluate the annotation consistency among multiple coders.
- Uses F1 score to measure agreement between each pair of coders
- Takes the average of all pairwise F1 scores as the overall consistency metric

In [29]:
base = importr("base")

# Import R irr package (for calculating inter-rater reliability)
irr = importr("irr")

In [30]:
def pairwise_f1_agreement(df):
    scores = []
    for i, j in combinations(df.columns, r=2):
        coder1 = df[i]
        coder2 = df[j]
        score = f1_score(coder1, coder2, average="binary")
        scores.append(score)

    return np.mean(scores)

In [35]:
def calculate_kappa(dataset, strategy, coders):
    dataset.valid_coders = coders
    dataset.strategy = strategy
    dataset.apply_label_strategy()

    df = dataset.df_labels.copy()
    raw_data = pd.DataFrame(df.labels.tolist(), columns=dataset.coders, index=df.id)
    f1_agreement = pairwise_f1_agreement(raw_data)
    raw_data = raw_data.to_numpy()


    numpy2ri.activate()  # 激活numpy到R的转换
    try:
        r_mat = ro.conversion.py2rpy(raw_data)

        out = irr.kappam_fleiss(r_mat, exact=False)
        kappa = np.array(out.rx("value"))[0][0]

        out = irr.agree(r_mat, tolerance=0)
        agree = np.array(out.rx("value"))[0][0]

        out = irr.kripp_alpha(base.t(r_mat), method="nominal")
        alpha = np.array(out.rx("value"))[0][0]
    finally:
        numpy2ri.deactivate()  # 确保在完成后停用转换

    return round(kappa, 3), round(alpha, 3), round(agree, 1), round(f1_agreement, 3)

This code implements a comprehensive inter-rater reliability assessment system for text classification tasks. The calculate_kappa function calculates multiple reliability metrics to evaluate the agreement between different coders (annotators) who have labeled the same text data.

In [32]:
coder_sets = {"all_coders": dataset.all_coders}

label_strategies = [
    strategies.PopEliteBin(),
    strategies.PopCentrBin(),
    strategies.IdeolLeftBin(),
    strategies.IdeolRightBin(),
]

# Kappa Values


In [36]:
rows = []
for strat in label_strategies:
    for set_name, coders in coder_sets.items():
        kappa, alpha, agree, f1 = calculate_kappa(dataset, strat, coders)
        rows.append((type(strat).__name__, set_name, kappa, f1, alpha, agree))

table = pd.DataFrame(
    rows,
    columns=[
        "strategy",
        "coder_set",
        "fleiss_kappa",
        "f1",
        "krippendorff_alpha",
        "agree",
    ],
).set_index(["strategy", "coder_set"])

In [37]:
# Reset index, drop unnecessary columns, and rename columns
tab = (
    table.reset_index()
    .drop(["coder_set", "krippendorff_alpha"], axis=1)
    .rename(
        {
            "strategy": "Label",
            "f1": "F1",
        },
        axis=1,
    )
)
tab.Label = tab.Label.replace(
    {
        "PopEliteBin": "Anti-Elitism",
        "PopCentrBin": "People-Centrism",
        "IdeolLeftBin": "Left-Wing Ideology",
        "IdeolRightBin": "Right-Wing Ideology",
    }
)
# get number of positive labels
dataset.strategy = strategies.MLMin1PopIdeol()
dataset.apply_label_strategy()

tab["N"] = 0
for i in range(0, 4):
    tab.loc[tab.index == i, "N"] = sum(labels[i] for labels in dataset.df_labels.vote)

# table footer
footer = (
    pd.Series(
        {
            "Label": "Total / Mean",
            "fleiss_kappa": round(tab["fleiss_kappa"].mean(), 3),
            "F1": round(tab["F1"].mean(), 3),
            "agree": round(tab["agree"].mean(), 3),
            "N": len(dataset.df_labels),
        },
    )
    .to_frame()
    .T
)
footer.index = [4]

tab = pd.concat([tab, footer])

tab["agree"] = [f"{val:.2f} \%" for val in tab["agree"]]
tab["fleiss_kappa"] = [f"{val:.3f}" for val in tab["fleiss_kappa"]]
tab["F1"] = [f"{val:.3f}" for val in tab["F1"]]

tab = tab[["Label", "N", "fleiss_kappa", "agree", "F1"]]

tab = tab.rename(
    {
        "agree": "Agreement",
        "fleiss_kappa": "Fleiss' $\kappa$",
    },
    axis=1,
)

tab.iloc[4] = [f"\\textbf{{{cell}}}" for cell in tab.iloc[4]]

tab

,Label,N,Fleiss' $\kappa$,Agreement,F1
0,Anti-Elitism,3236,0.410,65.80 \%,0.496
1,People-Centrism,1608,0.244,81.80 \%,0.262
2,Left-Wing Ideology,1393,0.355,84.50 \%,0.378
3,Right-Wing Ideology,773,0.364,91.60 \%,0.383
4,\textbf{Total / Mean},\textbf{8795},\textbf{0.343},\textbf{80.92 \%},\textbf{0.380}


# Get Number of Labels

In [39]:
# Exports the statistics table we just created to a LaTeX format file.
tab.to_latex(src.PATH / "results/tables/coder_agreement.tex", index=False, escape=False)